In [41]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = 'C:/Users/RaymondCarpenter/Documents/GitHub/14thstreetanalytics/nfl-draft/ol_sheet.csv'
data = pd.read_csv(file_path)
import pandas as pd
from scipy.stats import rankdata
from scipy.spatial.distance import cdist

# Assuming 'data' is your DataFrame loaded from the provided CSV

# Define the numerical columns we're interested in
numerical_cols = [
    'Height (in)', 'Weight (lbs)', 'Hand Size (in)', 'Arm Length (in)',
    '40 Yard', 'Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle', '3Cone', '60Yd Shuttle'
]

# Calculate percentiles for each numeric column
for col in numerical_cols:
    data[f'{col} Percentile'] = rankdata(data[col], method='average') / len(data[col])

# Separate the dataset into 2024 and others based on the calculated percentiles
data_2024 = data[data['Year'] == 2024]
data_others = data[data['Year'] != 2024]

# Extract the percentile columns for comparison
percentile_cols = [f'{col} Percentile' for col in numerical_cols]
data_2024_percentiles = data_2024[percentile_cols].fillna(0.5)  # Using 0.5 to represent missing data as median performance
data_others_percentiles = data_others[percentile_cols].fillna(0.5)

# Calculate Euclidean distances between 2024 players and others based on percentiles
distances = cdist(data_2024_percentiles, data_others_percentiles)

# For each 2024 player, find the index of the most similar player in the non-2024 dataset
most_similar_indices = distances.argmin(axis=1)

# Map the indices to player names
most_similar_names = data_others.iloc[most_similar_indices]['Name'].values

# Create a DataFrame for easy viewing
similarity_results = pd.DataFrame({
    '2024 Player': data_2024['Name'].values,
    'Most Similar Player': most_similar_names
})

# Display the results
print(similarity_results)


                 2024 Player Most Similar Player
0                    Joe Alt       Landon Turner
1              Graham Barton         Kalon Davis
2               Andrew Coker       Landon Turner
3                 Frank Crum       Syrus Tuitele
4               Anim Dankwah    Desmond Harrison
..                       ...                 ...
65               Andrew Raym         Netane Muti
66                Nick Samac           Coy Cronk
67             Jalen Sundell           Coy Cronk
68            Charles Turner         Jemar Clark
69  Sedrick Van Pran-Granger        Alex Forsyth

[70 rows x 2 columns]


In [43]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = 'C:/Users/RaymondCarpenter/Documents/GitHub/14thstreetanalytics/nfl-draft/ol_sheet.csv'
data = pd.read_csv(file_path)

import pandas as pd
from scipy.stats import rankdata
from scipy.spatial.distance import cdist

# Define the numerical columns relevant for offensive linemen
numerical_cols = [
    'Height (in)', 'Weight (lbs)', 'Hand Size (in)', 'Arm Length (in)',
    '40 Yard', 'Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle', '3Cone', '60Yd Shuttle'
]

# Calculate percentiles for each numeric column across all players
for col in numerical_cols:
    data[f'{col} Percentile'] = rankdata(data[col], method='average') / len(data[col])

# Select percentile columns for comparison
percentile_cols = [f'{col} Percentile' for col in numerical_cols]
data_percentiles = data[percentile_cols].fillna(0.5)  # Assuming median for missing

# Input for a specific OL player's name from the user
player_name = input("Enter the name of the player to compare: ")

try:
    # Find the player's data
    player_data = data[data['Name'].str.lower() == player_name.lower()]
    
    if player_data.empty:
        raise ValueError("Player not found. Please check the name and try again.")
    
    player_percentiles = player_data[percentile_cols].values

    # Calculate Euclidean distances from this player to all others
    distances = cdist(player_percentiles, data_percentiles, metric='euclidean')

    # Exclude the player themselves from the comparison
    distances[0, player_data.index[0]] = float('inf')

    # Find the most similar player
    most_similar_index = distances.argmin()
    most_similar_player = data.iloc[most_similar_index]
    most_similar_player_name = most_similar_player['Name']

    # Display the most similar player name
    print(f"\nThe most similar player to {player_name} is {most_similar_player_name}.\n")

    # Prepare and display a side-by-side comparison of their metrics
    comparison_metrics = data.loc[player_data.index[0], numerical_cols].to_frame(player_name)
    comparison_metrics[most_similar_player_name] = data.loc[most_similar_index, numerical_cols].values

    print(comparison_metrics)

except ValueError as e:
    print(e)



The most similar player to Joe Alt is Trevor Penning.

                Joe Alt Trevor Penning
Height (in)       80.75          79.13
Weight (lbs)        321            325
Hand Size (in)     10.0          10.13
Arm Length (in)   34.25          34.25
40 Yard            5.05           4.89
Bench Press         NaN            NaN
Vert Leap (in)     28.0           28.0
Broad Jump (in)   112.0          111.0
Shuttle            4.51           4.62
3Cone              7.31           7.25
60Yd Shuttle        NaN            NaN
